In [1]:
import sqlite3

In [2]:
conn=sqlite3.connect('example.db')

In [ ]:
conn

In [ ]:
conn.execute("""CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY,
              name TEXT, 
            age INTEGER
             salary REAL not null)""") 

In [ ]:
conn.execute("""CREATE TABLE IF NOT EXISTS employes (id INTEGER PRIMARY KEY,
              first_name TEXT, 
            age INTEGER,
             email REAL not null)""") 

In [ ]:

conn.execute("""CREATE TABLE IF NOT EXISTS customer (
                          id INTEGER PRIMARY KEY,
                          first_name TEXT NOT NULL,
                          last_name TEXT NOT NULL,
                          age INTEGER,
                          email TEXT NOT NULL,
                          phone_number TEXT,
                          address TEXT
                         )""")

In [ ]:
# Drop the existing 'users' table
conn.execute("DROP TABLE IF EXISTS users")

# Create the 'users' table with the correct schema
conn.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT, 
    age INTEGER,
    salary REAL NOT NULL
)
""")

# Insert data into the 'users' table
conn.execute("""
INSERT INTO users (id, name, age, salary) 
VALUES 
(1, 'John Doe', 30, 50000.0),
(2, 'Jane Doe', 25, 60000.0),
(3, 'Alice Smith', 40, 70000.0),
(4, 'Bob Brown', 35, 55000.0),
(5, 'Charlie White', 28, 48000.0)
""")

In [ ]:
conn.execute("""
            INSERT INTO employes (id, first_name, age, email) 
            VALUES 
            (1, 'Emily', 29, 'emily@example.com'),
            (2, 'Michael', 35, 'michael@example.com'),
            (3, 'Sarah', 42, 'sarah@example.com'),
            (4, 'David', 31, 'david@example.com'),
            (5, 'Laura', 27, 'laura@example.com')
            """)

In [ ]:
conn.execute("""
                        INSERT INTO customer(id, first_name, last_name, age, email, phone_number, address) 
                        VALUES 
                        (1, 'John', 'Doe', 30, 'john.doe@example.com', '123-456-7890', '123 Elm Street'),
                        (2, 'Jane', 'Smith', 25, 'jane.smith@example.com', '987-654-3210', '456 Oak Avenue'),
                        (3, 'Alice', 'Johnson', 40, 'alice.johnson@example.com', '555-555-5555', '789 Pine Road'),
                        (4, 'Bob', 'Williams', 35, 'bob.williams@example.com', '111-222-3333', '321 Maple Lane'),
                        (5, 'Charlie', 'Brown', 28, 'charlie.brown@example.com', '444-444-4444', '654 Cedar Drive')
                        """)

In [111]:
cursor = conn.execute("SELECT * FROM employes")
for row in cursor:
    print(row)

(1, 'Emily', 29, 'emily@example.com')
(2, 'Michael', 35, 'michael@example.com')
(3, 'Sarah', 42, 'sarah@example.com')
(4, 'David', 31, 'david@example.com')
(5, 'Laura', 27, 'laura@example.com')


In [ ]:
# Install the langchain-community package
%pip install langchain-community

# Import the SQLDatabase module
# from langchain_community.utilities import SQLDatabase

In [39]:
from langchain_community.utilities import SQLDatabase

In [40]:
db= SQLDatabase.from_uri("sqlite:///example.db")

In [ ]:
db.get_usable_table_names()

In [ ]:
db.get_table_info(['customer'])


In [ ]:
db.get_table_info("userss")

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [52]:
from langchain_groq import ChatGroq
llm= ChatGroq(model="llama-3.1-8b-instant")

In [ ]:
messages = [
    ("system", "You are a Chatbot that can answer questions"),
    ("human", "who won IPL 2022?"),
]
llm.invoke(messages)

In [63]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [ ]:
toolkit= SQLDatabaseToolkit(db=db, llm=llm)
tools=toolkit.get_tools()
for tool in tools:
    print(tool.name, tool.description)

In [78]:
list_table=next((tool for tool in tools if tool.name == "sql_db_list_tables"),None)
list_table.invoke('{customer}')

In [ ]:
schema=next((tool for tool in tools if tool.name == "sql_db_schema"),None)
schema.invoke('customer')

In [ ]:
@tool
def db_query_tool(query : str)-> str:
    """Execute the SQL Query and return the result.
    If the query is not valid, return an error message.
    In case of an error, user is advised to rewrite the query.
    """
    try:
        result = db.run_no_throw(query)
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}. Please rewrite the query."
    

In [ ]:
db.run("SELECT * FROM customer")